In [110]:
# Required libraries
import pandas as pd
import numpy as np
import scipy as sp
import math
from scipy import stats

In [111]:
# Load data set
df = pd.read_csv("FinDataSet-1.csv")
df.index = pd.to_datetime(df["Date"], format="%d/%m/%Y")
df.head()

,Date,CBA,MQG
Date,,,
2013-01-02,2/1/2013,73.134,38.539
2013-01-03,3/1/2013,72.636,37.851
2013-01-04,4/1/2013,72.188,38.235
2013-01-07,7/1/2013,70.754,42.394
2013-01-08,8/1/2013,71.362,43.368


In [112]:
# Current values of financial variables
current_values = df.tail(1).to_numpy()
CBA_price = current_values[0,1]
MQG_price = current_values[0,1]

In [113]:
# Drop other columns
df = df.drop(["Date"], axis=1)

# Calculate the daily log-return:
for col in df.columns.values:
    df.loc[:,col] = np.log(df.loc[:,col]) - np.log(df.loc[:,col].shift(1))

df = df.dropna()
df.head()

,CBA,MQG
Date,,
2013-01-03,-0.006833,-0.018013
2013-01-04,-0.006187,0.010094
2013-01-07,-0.020065,0.103256
2013-01-08,0.008556,0.022715
2013-01-09,-0.019299,0.011486


In [114]:
# Covariance matrix of (daily) log returns
daily_log_ret_cov = np.cov(df, rowvar = False)
daily_log_ret_cov

CBA_ann_vol = np.sqrt(daily_log_ret_cov[0,0] * 250)
MQG_ann_vol = np.sqrt(daily_log_ret_cov[1,1] * 250)

## Portfolio Value

### The entire portfolio value:

$$ V_t = V_{\text{CBA},t} + V_{\text{MQG},t} $$

Where each position value as follows:
$$\begin{align*}
V_{\text{CBA}, t} &= V_{\text{call}, 80} \left( t, S_{CBA,t}, r_t^{(6m)} \right) + V_{\text{call}, 110} \left( t, S_{CBA,t}, r_t^{(6m)} \right) - 2 \times V_{\text{call}, 100} \left( t, S_{CBA,t}, r_t^{(6m)} \right) \\[5pt]
V_{\text{MQG}, t} &= V_{\text{put}, 150} \left( t, S_{MQG,t}, r_t^{(6m)} \right) + V_{\text{call}, 220} \left( t, S_{MQG,t}, r_t^{(6m)} \right)
\end{align*}$$

## 1. Use the Black-Scholes-Merton formula to determine the mark-to-market value

In [115]:
# Pricing function for European calls and puts
def BSprice(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    
    # Calculate option prices
    if option_type == 'call':
        price = (np.exp(-q * tau) * s * sp.stats.norm.cdf(d1, 0.0, 1.0) \
                 - np.exp(-r * tau) * K * sp.stats.norm.cdf(d2, 0.0, 1.0))
        
    if option_type == 'put':
        price = (np.exp(-r * tau) * K * sp.stats.norm.cdf(-d2, 0.0, 1.0) \
                 - np.exp(-q * tau) * s * sp.stats.norm.cdf(-d1, 0.0, 1.0))
        
    return price

In [116]:
# Assign values:
rfr = 0.0267
tau = 0.5
dy = 0

# Butterfly spread CBA:
CBA_l_call_strike_80 = 80
CBA_l_call_strike_110 = 110
CBA_s_call_strike_100_x2 = 100

# Strangle MQG:
MQG_l_put_strike_150 = 150
MQG_l_call_strike_220 = 220

# Calculate the mark-to-market value of the butterfly spread:
bs_CBA_long_1 = BSprice(CBA_price, CBA_l_call_strike_80, time_to_maturity, risk_free_rate, dividend_yield, CBA_ann_vol, "call")
bs_CBA_long_2 = BSprice(CBA_price, CBA_l_call_strike_110, time_to_maturity, risk_free_rate, dividend_yield, CBA_ann_vol, "call")
bs_CBA_short_3 = - 2 * BSprice(CBA_price, CBA_s_call_strike_100_x2, time_to_maturity, risk_free_rate, dividend_yield, CBA_ann_vol, "call")
bs_value = bs_CBA_long_1 + bs_CBA_long_2 + bs_CBA_short_3

# Calculate the mark-to-market value of the strangle:
st_MQG_long_1 = BSprice(MQG_price, MQG_l_put_strike_150, time_to_maturity, risk_free_rate, dividend_yield, MQG_ann_vol, "put")
st_MQG_long_2 = BSprice(MQG_price, MQG_l_call_strike_220, time_to_maturity, risk_free_rate, dividend_yield, MQG_ann_vol, "call")
st_value = st_MQG_long_1 + st_MQG_long_2

In [117]:
print(bs_CBA_long_1)
print(bs_CBA_long_2)
print(bs_CBA_short_3)
print(bs_value)

print(st_MQG_long_1)
print(st_MQG_long_2)
print(st_value)

57.80795551621756
50.10402500659926
-104.87192106017369
3.0400594626431285
47.383777660490765
7.894402875922076e-05
47.38385660451952


## 2. Implement meethods from (M1) to (M5)

### Risk Factor Mapping

We introduce the following notation:

$$
\mathbf{Z}_t := \begin{bmatrix} Z_{1,t} \\ Z_{2,t} \end{bmatrix} = \begin{bmatrix} \ln S_{CBA,t} \\ \ln S_{MQG,t}  \end{bmatrix}, \quad \mathbf{X}_{t+1} := \mathbf{Z}_{t+1} - \mathbf{Z}_t = \begin{bmatrix} \ln S_{CBA,t+1} - \ln S_{CBA,t} \\ \ln S_{MQG,t+1} - \ln S_{MQG,t} \end{bmatrix}
$$

A mapping, therefore, of the risk factor vector Zt to the current portfolio value is
$$\begin{align*}
V_t &= g(t, \mathbf{Z}_t) \\[5pt]
&:= V_{\text{CBA}} \left( t, e^{Z_{1,t}} \right) + V_{\text{MQG}} \left( t, e^{Z_{2,t}} \right).
\end{align*}$$

### 2.1. (M1) Analytical delta-normal approach

In [131]:
# Create dataframe to store value
portfolio_risk_measures = pd.DataFrame(columns=['method', 'type', 'conf_level', 'risk_measure'])

# Confidence level
conf_level = [0.9, 0.95, 0.99]

In [119]:
# Function to calculate option delta under the Black-Scholes-Merton model
def BSdelta(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility, option_type):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Calculate d1 and d2
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    
    # Calculate option delta
    if option_type == 'call':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0)
    
    if option_type == 'put':
        value = sp.stats.norm.cdf(d1, 0.0, 1.0) - 1
    
    return value    

In [120]:
# Call Option Risk Factor Exposures
# - Call delta
call_delta_CBA_80 = BSdelta(spot_price = CBA_price, strike_price = CBA_l_call_strike_80, time_to_maturity = tau, 
                     risk_free_rate = rfr, dividend_yield = dy, volatility = CBA_ann_vol, option_type = 'call')
call_delta_CBA_110 = BSdelta(spot_price = CBA_price, strike_price = CBA_l_call_strike_110, time_to_maturity = tau, 
                     risk_free_rate = rfr, dividend_yield = dy, volatility = CBA_ann_vol, option_type = 'call')
call_delta_CBA_100_x2 = BSdelta(spot_price = CBA_price, strike_price = CBA_s_call_strike_100_x2, time_to_maturity = tau, 
                     risk_free_rate = rfr, dividend_yield = dy, volatility = CBA_ann_vol, option_type = 'call')

call_delta_MQG_220 = BSdelta(spot_price = MQG_price, strike_price = MQG_l_call_strike_220, time_to_maturity = tau, 
                     risk_free_rate = rfr, dividend_yield = dy, volatility = MQG_ann_vol, option_type = 'call')

# Put Option Risk Factor Exposures
# - Put delta
put_delta_MQG_150 = BSdelta(spot_price = MQG_price, strike_price = MQG_l_call_strike_220, time_to_maturity = tau, 
                     risk_free_rate = rfr, dividend_yield = dy, volatility = MQG_ann_vol, option_type = 'put')

# CBA Risk Factor Exposures
CBA_exposure1 = (call_delta_CBA_80 + call_delta_CBA_110 - 2*call_delta_CBA_100_x2) * CBA_price
CBA_exposure2 = 0
CBA_exposure = np.array([CBA_exposure1, CBA_exposure2])

# MQG Risk Factor Exposures
MQG_exposure1 = 0
MQG_exposure2 = (call_delta_MQG_220 - put_delta_MQG_150) * MQG_price
MQG_exposure = np.array([MQG_exposure1, MQG_exposure2])

In [121]:
# Mean log-returns
daily_log_ret_mean = np.mean(df, axis = 0)
daily_log_ret_mean

# 10-day log-returns mean
log_ret_mean_10days = daily_log_ret_mean * 10

In [122]:
# 1-day volatility
CBA_1day_vol = np.sqrt(daily_log_ret_cov[0,0])
MQG_1day_vol = np.sqrt(daily_log_ret_cov[1,1])

# 10-day volatility
log_ret_cov_10days = daily_log_ret_cov * np.sqrt(10)
CBA_10days_vol = np.sqrt(daily_log_ret_cov[0,0] * 10)
MQG_10days_vol = np.sqrt(daily_log_ret_cov[1,1] * 10)

In [123]:
# Functions for VaR and ES under the variance-covariance approach
def VaR_ES_vcv(constant_loss, risk_exposure, risk_factor_mean, risk_factor_cov_mat, conf_level):

    # Shorthand notation for inputs
    c = constant_loss
    b = risk_exposure
    mu = risk_factor_mean
    Sigma = risk_factor_cov_mat
    alpha = conf_level

    # Compute mean and variance of loss
    loss_mean = -c - np.transpose(b) @ mu
    loss_var = np.transpose(b) @ Sigma @ b

    # Compute VaR
    VaR = loss_mean + np.sqrt(loss_var) * stats.norm.ppf(alpha, loc = 0.0, scale = 1.0)
    VaR = max(VaR, 0)

    # Compute ES
    ES = loss_mean + np.sqrt(loss_var) * stats.norm.pdf(stats.norm.ppf(alpha, loc = 0.0, scale = 1.0), loc = 0.0, scale = 1.0) / (1 - alpha)
    ES = max(ES, 0)

    # Return output
    return np.array([VaR, ES])

In [132]:
# Portfolio Risk Measures
portfolio_exposure = CBA_exposure + MQG_exposure

# Compute 1-day VaR and ES
new_rows = []
for i in conf_level:
    risk_measure = VaR_ES_vcv(0, portfolio_exposure, daily_log_ret_mean, daily_log_ret_cov, i)
    new_row = {'method': 'M1', 'type': '1-day', 'conf_level': i, 'risk_measure': risk_measure}
    new_rows.append(new_row)

# Add new rows to the DataFrame using pd.concat()
portfolio_risk_measures = pd.concat([portfolio_risk_measures, pd.DataFrame(new_rows)], ignore_index=True)

# Compute 10-day VaR and ES
new_rows = []
for i in conf_level:
    risk_measure = VaR_ES_vcv(0, portfolio_exposure, log_ret_mean_10days, log_ret_cov_10days, i)
    new_row = {'method': 'M1', 'type': '10-day', 'conf_level': i, 'risk_measure': risk_measure}
    new_rows.append(new_row)

# Add new 10-day rows to the DataFrame
portfolio_risk_measures = pd.concat([portfolio_risk_measures, pd.DataFrame(new_rows)], ignore_index=True)

print(portfolio_risk_measures)

  method    type  conf_level                              risk_measure
0     M1   1-day        0.90   [2.1169787242342273, 2.923032381427151]
1     M1   1-day        0.95  [2.7355281834212257, 3.4469652976184495]
2     M1   1-day        0.99   [3.8958240990122137, 4.472769890158901]
3     M1  10-day        0.90   [3.2304923755592996, 4.663880997532049]
4     M1  10-day        0.95     [4.33044614292225, 5.595580114636682]
5     M1  10-day        0.99    [6.393776479169965, 7.419747300274736]


/var/folders/8v/_wg4l14d02v5d6jkm_cpp_6w0000gn/T/ipykernel_3087/3052406946.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  portfolio_risk_measures = pd.concat([portfolio_risk_measures, pd.DataFrame(new_rows)], ignore_index=True)


### 2.2. (M2) Historical simulation approach with a delta-gamma approximation

In [ ]:
# Function to calculate option gamma under the Black-Scholes-Merton model
def BSgamma(spot_price, strike_price, time_to_maturity, risk_free_rate, dividend_yield, volatility):
    
    # Use mathematical notation for function inputs
    s = spot_price
    K = strike_price
    tau = time_to_maturity # (T-t) in notes
    r = risk_free_rate
    q = dividend_yield
    sigma = volatility
    
    # Compute option gamma (same for calls and puts)
    d1 = (np.log(s / K) + (r - q + 0.5 * sigma ** 2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    
    value = sp.stats.norm.pdf(d1, 0.0, 1.0) / (s * sigma * np.sqrt(tau))
    
    return value